<div class="alert alert-block alert-info" style="margin-top: 20px">

      
| Name | Description | Date
| :- |-------------: | :-:
|Reza Hashemi| Multi Layer Perceptrons - 3rd  | On 23rd of August 2019 | width="750" align="center"></a></p>
</div>

# Multi Layer Perceptrons
- CIFAR-10 image classification with deeper MLP
  - Last time, we have attempted to classify images in the CIFAR-10 dataset with MLP. However, we were able to obtain accuracy of only 0.369.
  - Here, we attempt to gain higher accuracy by changing the MLP architecture and hyperparameters.

In [0]:
!pip3 install torch torchvision

In [0]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.0.0'

## 1. Import & process dataset
- CIFAR10 dataset can be downloaded by ```torchvision```
  - [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)

In [0]:
from torchvision import datasets
import torchvision.transforms as transforms

train_dataset = datasets.CIFAR10(root = "/", train = True, download = True, transform = transforms.ToTensor())
test_dataset = datasets.CIFAR10(root = "/", train = False, download = True, transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


## 2. Creating MLP model and training

- Create and train MLP with more hidden layers and advanced settings

![](http://cs231n.github.io/assets/nn1/neural_net.jpeg)

In [0]:
# create data loaders 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128, shuffle = False)

In [0]:
# create MLP with one hidden layer
class net(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(net, self).__init__()
    self.input_dim = input_dim
    self.dense1 = nn.Linear(input_dim, hidden_dim[0])      # first hidden layer
    self.dense2 = nn.Linear(hidden_dim[0], hidden_dim[1])  # second hidden layer
    self.dense3 = nn.Linear(hidden_dim[1], hidden_dim[2])  # third hidden layer
    self.dense4 = nn.Linear(hidden_dim[2], output_dim)     
    self.relu = nn.ReLU()
     
  def forward(self, x):
    x = x.view(x.size(0), self.input_dim)
    x = self.dense1(x)
    x = self.relu(x)
    x = self.dense2(x)
    x = self.relu(x)
    x = self.dense3(x)
    x = self.relu(x)
    x = self.dense4(x)
    return x

In [0]:
# hyperparameters
DEVICE = torch.device('cuda')
INPUT_DIM = 32 * 32 * 3
HIDDEN_DIM = (100, 100, 100)    # 3 hidden dimensions for 3 layers
OUTPUT_DIM = 10
LEARNING_RATE = 1e-3
NUM_EPOCHS = 30              # increase number of epochs

In [0]:
model = net(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
criterion = nn.CrossEntropyLoss()   # do not need softmax layer when using CEloss criterion
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [0]:
# training for NUM_EPOCHS
for i in range(NUM_EPOCHS):
  temp_loss = []
  for (x, y) in train_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = model(x)
    loss = criterion(outputs, y)
    temp_loss.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print("Loss at {}th epoch: {}".format(i, np.mean(temp_loss)))

Loss at 0th epoch: 1.89627995271512
Loss at 1th epoch: 1.693795374592247
Loss at 2th epoch: 1.6212971856831895
Loss at 3th epoch: 1.5668071813290687
Loss at 4th epoch: 1.5262251299665408
Loss at 5th epoch: 1.4933200495322343
Loss at 6th epoch: 1.4687112606394932
Loss at 7th epoch: 1.4454451488411946
Loss at 8th epoch: 1.4289868929806877
Loss at 9th epoch: 1.408607015829257
Loss at 10th epoch: 1.394239951582516
Loss at 11th epoch: 1.3733605451291175
Loss at 12th epoch: 1.3631285389366052
Loss at 13th epoch: 1.343611014011266
Loss at 14th epoch: 1.3399841968360764
Loss at 15th epoch: 1.3238185805737819
Loss at 16th epoch: 1.3146519923149167
Loss at 17th epoch: 1.3017306763802647
Loss at 18th epoch: 1.2912272934413627
Loss at 19th epoch: 1.2836608034570505
Loss at 20th epoch: 1.2702278880512012
Loss at 21th epoch: 1.2586160532348907
Loss at 22th epoch: 1.2533685801279209
Loss at 23th epoch: 1.2392126495575966
Loss at 24th epoch: 1.2397530463040638
Loss at 25th epoch: 1.2295762690741692
Lo

## 3. Evaluation
- Evaluate the trained MLP model with ROC AUC score 
  - Store probability of each instance to a list and compare it with true y label

In [0]:
y_pred, y_true = [], []
with torch.no_grad():
  for x, y in test_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = F.softmax(model(x)).max(1)[-1]       # predicted label
    y_true += list(y.cpu().numpy())                # true label
    y_pred += list(outputs.cpu().numpy())   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [0]:
# evaluation result
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.504